In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cardataset/data.csv


In [2]:
df_old = pd.read_csv("/kaggle/input/cardataset/data.csv")
df_old.head()

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500


In [3]:
features = ['Make',
'Model',
'Year',
'Engine HP',
'Engine Cylinders',
'Transmission Type',
'Vehicle Style',
'highway MPG',
'city mpg',
'MSRP']

In [4]:
df = df_old.loc[:,features]

In [5]:
df.columns = df.columns.str.replace(' ', '_').str.lower()
df = df.rename(columns={"msrp": "price"})
df = df.fillna(0)
df.head()

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg,price
0,BMW,1 Series M,2011,335.0,6.0,MANUAL,Coupe,26,19,46135
1,BMW,1 Series,2011,300.0,6.0,MANUAL,Convertible,28,19,40650
2,BMW,1 Series,2011,300.0,6.0,MANUAL,Coupe,28,20,36350
3,BMW,1 Series,2011,230.0,6.0,MANUAL,Coupe,28,18,29450
4,BMW,1 Series,2011,230.0,6.0,MANUAL,Convertible,28,18,34500


In [6]:
df.transmission_type.value_counts() #Q1 - Automatic

transmission_type
AUTOMATIC           8266
MANUAL              2935
AUTOMATED_MANUAL     626
DIRECT_DRIVE          68
UNKNOWN               19
Name: count, dtype: int64

In [7]:
categorical = ['make','model','transmission_type','vehicle_style']
numerical = ['year','engine_hp','engine_cylinders','highway_mpg','city_mpg']
cor_mtrx = pd.DataFrame(columns=numerical,index = numerical)
for ft1 in numerical:
    cor_score = df[numerical].corrwith(df[ft1])
    cor_mtrx[ft1] = cor_score.values
cor_mtrx #Q2 - highway_mpg and city_mpg

,year,engine_hp,engine_cylinders,highway_mpg,city_mpg
year,1.000000,0.338714,-0.040708,0.258240,0.198171
engine_hp,0.338714,1.000000,0.774851,-0.415707,-0.424918
engine_cylinders,-0.040708,0.774851,1.000000,-0.614541,-0.587306
highway_mpg,0.258240,-0.415707,-0.614541,1.000000,0.886829
city_mpg,0.198171,-0.424918,-0.587306,0.886829,1.000000


In [8]:
above_average = (df.price >= df.price.mean())
above_average = above_average.astype(int)
df['above_average'] = above_average
del df['price']
df.head()

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg,above_average
0,BMW,1 Series M,2011,335.0,6.0,MANUAL,Coupe,26,19,1
1,BMW,1 Series,2011,300.0,6.0,MANUAL,Convertible,28,19,1
2,BMW,1 Series,2011,300.0,6.0,MANUAL,Coupe,28,20,0
3,BMW,1 Series,2011,230.0,6.0,MANUAL,Coupe,28,18,0
4,BMW,1 Series,2011,230.0,6.0,MANUAL,Convertible,28,18,0


In [9]:
from sklearn.model_selection import train_test_split

In [10]:
df_full_train, df_test = train_test_split(df, test_size = 0.2, random_state = 42)

In [11]:
df_train, df_val = train_test_split(df_full_train, test_size = 0.25, random_state = 42)

In [12]:
df_train = df_train.reset_index(drop = True)
df_val = df_val.reset_index(drop = True)
df_test = df_test.reset_index(drop = True)

In [13]:
y_train = df_train.above_average.values
y_val = df_val.above_average.values
y_test = df_test.above_average.values

In [14]:
del df_train['above_average']
del df_val['above_average']
del df_test['above_average']

In [15]:
from sklearn.metrics import mutual_info_score

In [16]:
def mutual_info_above_avg_score(series):
    return mutual_info_score(series, df_full_train.above_average)

In [17]:
mi = df_full_train[categorical].apply(mutual_info_above_avg_score)
mi.sort_values(ascending=False) #Q3 - transmission_type

model                0.460994
make                 0.238724
vehicle_style        0.083390
transmission_type    0.020884
dtype: float64

In [18]:
from sklearn.feature_extraction import DictVectorizer

In [19]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [20]:
from sklearn.linear_model import LogisticRegression

In [21]:
from sklearn.base import clone

In [22]:
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(C=10, max_iter=1000, random_state=42, solver='liblinear')

In [23]:
y_pred = model.predict_proba(X_val)[:, 1]

In [24]:
avg_decision = (y_pred >= 0.5)

In [25]:
Z = (y_val == avg_decision).mean() #Q4 = 0,95

In [26]:
dv = DictVectorizer(sparse=False)
count = 0
scoring = []
def vectorize_train(df, dv, sets):
    train_dict = df[sets].to_dict(orient='records')
    X_train = dv.fit_transform(train_dict)
    return X_train
def vectorize_val(df, dv, sets):
    val_dict = df[sets].to_dict(orient='records')
    X_val = dv.transform(val_dict)
    return X_val  
   
features_order = []    
total_fts = []
for feature in (categorical + numerical):
    feats = (categorical + numerical).copy()
    new_ft = [i for i in feats if i  != str(feature)]
    total_fts.append(new_ft)
    features_order.append(feature)
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
for sets in total_fts:
    X_train = vectorize_train(df_train, dv, sets)
    X_val = vectorize_val(df_val, dv, sets)
    model.fit(X_train, y_train)
    
    y_pred = model.predict_proba(X_val)[:, 1]
    avg_decision = (y_pred >= 0.5)
    score = (y_val == avg_decision).mean()
    
    scoring.append(score)
    
final_score = pd.DataFrame(scoring,features_order,columns = ['score'])


In [27]:

final_score['diff'] =  Z - final_score['score'] 

In [28]:
final_score.sort_values('diff',ascending = True) #Q5 - year

,score,diff
make,0.948804,-0.014268
year,0.948384,-0.013848
engine_cylinders,0.947125,-0.012589
highway_mpg,0.946706,-0.012170
city_mpg,0.945867,-0.011330
transmission_type,0.945027,-0.010491
engine_hp,0.934956,-0.000420
vehicle_style,0.932018,0.002518
model,0.916492,0.018044


In [29]:
df6 = df_old.loc[:,features]
df6.columns = df6.columns.str.replace(' ', '_').str.lower()
df6 = df6.rename(columns={"msrp": "price"})
df6 = df6.fillna(0)
df6['price'] = df6['price'].apply(np.log1p)
df6

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg,price
0,BMW,1 Series M,2011,335.0,6.0,MANUAL,Coupe,26,19,10.739349
1,BMW,1 Series,2011,300.0,6.0,MANUAL,Convertible,28,19,10.612779
2,BMW,1 Series,2011,300.0,6.0,MANUAL,Coupe,28,20,10.500977
3,BMW,1 Series,2011,230.0,6.0,MANUAL,Coupe,28,18,10.290483
4,BMW,1 Series,2011,230.0,6.0,MANUAL,Convertible,28,18,10.448744
...,...,...,...,...,...,...,...,...,...,...
11909,Acura,ZDX,2012,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,10.739024
11910,Acura,ZDX,2012,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,10.945018
11911,Acura,ZDX,2012,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,10.832122
11912,Acura,ZDX,2013,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,10.838031


In [30]:
df_full_train, df_test = train_test_split(df6, test_size = 0.2, random_state = 42)

In [31]:
df_train, df_val = train_test_split(df_full_train, test_size = 0.25, random_state = 42)

In [32]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [33]:
from sklearn.linear_model import Ridge

In [34]:
y_train = df_train.price.values
y_val = df_val.price.values

In [35]:
del df_train['price']
del df_val['price']

In [36]:
def rmse(y, y_train):
    error = y - y_train
    se = error**2
    mse = se.mean()
    return np.sqrt(mse)

In [37]:
for alph in [0, 0.01, 0.1, 1, 10]:
    modl = Ridge(alpha = alph, solver='sag', random_state=42,tol=0.001)
    modl.fit(X_train, y_train)
    
    y_pred = modl.predict(X_val)
    print(f' for alpha = {alph} : score = {rmse(y_pred,y_val)}')

 for alpha = 0 : score = 0.4940098299300916
 for alpha = 0.01 : score = 0.49400998387687584
 for alpha = 0.1 : score = 0.4940113693565554
 for alpha = 1 : score = 0.4940252207146738
 for alpha = 10 : score = 0.49425040974751505
